<a href="https://colab.research.google.com/github/CyberMetrics/Prototypes/blob/main/Agent01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
from google.colab import drive;drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
import time
import random
import json
import pandas as pd
from datetime import datetime, timedelta
from typing import Dict, Any, List
import os # Necessary for os.fsync()

# --- Configuration: Define the Drive Paths ---
# REPLACE 'Capstone Mark-01' with your actual folder path if different
DRIVE_OUTPUT_PATH = '/content/drive/MyDrive/Capstone Mark-01/live_security_feed.json'

# --- Agent Core Logic ---

class CustomAgentLogGenerator:
    """Agent core logic for assigning custom rule metrics."""

    def __init__(self, agent_id: str, agent_name: str, system_os: str):
        self.agent_id = agent_id
        self.agent_name = agent_name
        self.system = system_os
        self.base_event_levels = {
            "authentication_success": 3, "system_startup": 2, "file_integrity_change": 7,
            "network_connection": 4, "port_scan_probe": 8, "brute_force_attempt": 10
        }
        self.CUSTOM_RULE_ID_BASE = 900000

    def _generate_rule_id_and_level(self, event_type: str, severity: str) -> Dict[str, int]:
        base_level = self.base_event_levels.get(event_type, 5)
        severity_map = {'low': 0, 'medium': 2, 'high': 4, 'critical': 6}
        level_boost = severity_map.get(severity.lower(), 0)
        rule_level = min(15, base_level + level_boost)

        if event_type == "brute_force_attempt":
            rule_id = self.CUSTOM_RULE_ID_BASE + 101
        elif event_type == "file_integrity_change":
            rule_id = self.CUSTOM_RULE_ID_BASE + 205
        else:
            rule_id = self.CUSTOM_RULE_ID_BASE + random.randint(300, 399)
        return {"rule_id": rule_id, "rule_level": rule_level}

    def process_data_record(self, record: Dict[str, Any]) -> Dict[str, Any]:
        rule_data = self._generate_rule_id_and_level(
            event_type=record.get("event_type", "unknown_event"),
            severity=record.get("severity", "medium")
        )

        log_entry = {
            "timestamp": datetime.now().isoformat(timespec='milliseconds') + "+00:00",
            "srcip": record.get("srcip", "0.0.0.0"),
            "dstip": record.get("dstip", "0.0.0.0"),
            "rule_id": rule_data["rule_id"],
            "rule_level": rule_data["rule_level"],
            "severity": record.get("severity", "medium").lower(),
            "status": record.get("status", "success").lower(),
            "event_type": record.get("event_type", "unknown_event"),
            "action": record.get("action", "monitor"),
            "group": record.get("group", "[api_import]"),
            "message": record.get("message", f"Activity log for service {record.get('service')}."),
            "system": self.system,
            "agent_id": self.agent_id,
            "agent_name": self.agent_name,
            "raw_protocol": record.get("protocol", "tcp"),
            "raw_duration": record.get("duration", 0),
        }
        return log_entry

def generate_single_simulated_event(event_count: int) -> Dict[str, Any]:
    """Generates a single raw data event with a mix of risk levels."""
    is_high_risk = random.random() < 0.2

    if is_high_risk:
        if random.random() < 0.5:
            event_type = "brute_force_attempt"
            severity = "Critical"
            message = f"Multiple auth failures. Attempt {event_count}."
            srcip = "203.0.113.5"
            action = "block"
        else:
            event_type = "port_scan_probe"
            severity = "High"
            message = f"Suspicious probe on port {random.randint(1, 1024)}."
            srcip = "172.16.1.10"
            action = "detect"
    else:
        if random.random() < 0.5:
            event_type = "network_connection"
            severity = "Low"
            message = "Successful standard connection."
            srcip = f"192.168.1.{random.randint(10, 50)}"
            action = "allow"
        else:
            event_type = "authentication_success"
            severity = "Medium"
            message = "User 'svc_acc' logged in successfully."
            srcip = f"192.168.1.{random.randint(51, 99)}"
            action = "allow"

    data = {
        "service": "dynamic_service",
        "protocol": random.choice(["tcp", "udp"]),
        "duration": random.uniform(0.0, 5.0),
        "event_type": event_type,
        "severity": severity,
        "status": random.choice(["success", "failed"]),
        "srcip": srcip,
        "dstip": "10.0.0.1",
        "message": message,
        "action": action,
    }
    return data

# --- Main Agent Execution Block ---

 # --- Main Agent Execution Block (CORRECTED OUTPUT VISIBILITY) ---

if __name__ == "__main__":

    # Initialize Agent (Class definition must be included above this block)
    agent = CustomAgentLogGenerator(
        agent_id="001",
        agent_name="firewall-gateway",
        system_os="centos"
    )

    event_counter = 0

    print(f"--- 🔄 Continuous Agent Data Writer Started ---")
    print(f"Saving to GDrive Path: {DRIVE_OUTPUT_PATH}")
    print("Stop the kernel/execution to finish (Ctrl+C).")
    print("-" * 50)

    try:
        # Create the file if it doesn't exist
        if not os.path.exists(DRIVE_OUTPUT_PATH):
             with open(DRIVE_OUTPUT_PATH, 'w'):
                 pass

        while True:
            event_counter += 1

            raw_event = generate_single_simulated_event(event_counter)
            processed_log = agent.process_data_record(raw_event)

            # Write to Drive and force sync
            with open(DRIVE_OUTPUT_PATH, 'a', encoding='utf-8') as f:
                f.write(json.dumps(processed_log) + "\n")
                f.flush()
                os.fsync(f.fileno()) # Forces immediate disk write and sync

            # --- VISIBLE OUTPUT FIX: Print a new line for every event ---
            print(f"| {datetime.now().strftime('%H:%M:%S')} | Log #{event_counter:<4} | Type: {processed_log['event_type'][:15]:<15} | Severity: {processed_log['severity'].upper():<8} | SYNCED |")

            time.sleep(random.uniform(0.1, 0.5))

    except KeyboardInterrupt:
        print("\n\n--- 🛑 Agent Writer Interrupted by User ---")

    finally:
        print(f"✅ Total Events Generated and Saved to Drive: {event_counter}")

--- 🔄 Continuous Agent Data Writer Started ---
Saving to GDrive Path: /content/drive/MyDrive/Capstone Mark-01/live_security_feed.json
Stop the kernel/execution to finish (Ctrl+C).
--------------------------------------------------
| 19:48:49 | Log #1    | Type: port_scan_probe | Severity: HIGH     | SYNCED |
| 19:48:50 | Log #2    | Type: authentication_ | Severity: MEDIUM   | SYNCED |
| 19:48:50 | Log #3    | Type: authentication_ | Severity: MEDIUM   | SYNCED |
| 19:48:50 | Log #4    | Type: authentication_ | Severity: MEDIUM   | SYNCED |
| 19:48:51 | Log #5    | Type: network_connect | Severity: LOW      | SYNCED |
| 19:48:51 | Log #6    | Type: authentication_ | Severity: MEDIUM   | SYNCED |
| 19:48:51 | Log #7    | Type: port_scan_probe | Severity: HIGH     | SYNCED |
| 19:48:52 | Log #8    | Type: network_connect | Severity: LOW      | SYNCED |
| 19:48:52 | Log #9    | Type: network_connect | Severity: LOW      | SYNCED |
| 19:48:52 | Log #10   | Type: network_connect | Severity: